# Installation des dépendances nécessaires

In [ ]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from crewai import Agent, Task, Crew

# Connexion à l'API de Mistral

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "bDq0DcsTJjNX4XvCq7D6VRbnP9jZaDT5"
os.environ["OPENAI_API_BASE"] = "https://api.mistral.ai/v1"
os.environ["OPENAI_MODEL_NAME"] = "mistral-small"

# Connexion au drive

In [ ]:
from google.colab import drive
import os

# Monter Google Drive
drive.mount('/content/drive')

# Chemin du dossier à vérifier
folder_path = "/content/drive/My Drive/Test RAG"

# Vérifier l'existence du dossier
if os.path.exists(folder_path):
    print(f"Le dossier '{folder_path}' est bien présent.")
else:
    raise FileNotFoundError(f"Erreur : Le dossier '{folder_path}' n'existe pas dans votre Google Drive.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Le dossier '/content/drive/My Drive/Test RAG' est bien présent.


# Création des agents

In [ ]:
routing_agent = Agent(
    role="Agent de Routage des Questions",
    goal="Analyser chaque question des dossiers de subvention et solliciter les experts "
         "appropriés pour fournir des réponses précises et pertinentes, avec une priorité "
         "donnée à l'expertise du project_expert_agent.",
    backstory=(
        "Ancien coordinateur de projets en financement public, vous avez développé une expertise "
        "dans l’identification rapide des compétences nécessaires pour répondre efficacement aux "
        "exigences des dossiers de subvention. Fort de votre expérience dans la gestion d’équipes "
        "pluridisciplinaires, vous excellez à analyser les besoins d'information et à orienter "
        "les questions vers les experts les plus qualifiés. Vous comprenez que la présentation "
        "du projet est la pierre angulaire d’un dossier réussi et veillez donc à toujours "
        "impliquer en priorité le project_expert_agent avant de compléter avec d’autres analyses spécialisées."
    ),
    allow_delegation=True,
    verbose=True
)

In [ ]:
manager_agent = Agent(
    role="Manager de Rédaction de Subventions",
    goal="Orchestrer l'équipe d'experts et transformer leurs analyses "
         "en une réponse cohérente et percutante pour les dossiers de subvention",
    backstory=(
        "En tant qu'ancien responsable d'un cabinet de conseil spécialisé dans "
        "les subventions publiques, vous avez piloté avec succès plus de 500 "
        "dossiers de financement. Votre expertise en coordination d'équipes "
        "pluridisciplinaires et votre capacité à synthétiser des analyses "
        "complexes font de vous le chef d'orchestre idéal de ce processus. "
        "Vous excellez dans l'art de transformer des contributions techniques "
        "en argumentaires convaincants, tout en maintenant une vision globale "
        "cohérente. Votre expérience vous permet d'anticiper les attentes des "
        "financeurs et de guider efficacement chaque expert pour obtenir les "
        "informations les plus pertinentes à la construction du dossier."
    ),
    allow_delegation=True,
    verbose=True
)

In [ ]:
diagnostic_analyst_agent = Agent(
    role="Analyste Territorial",
    goal="Évaluer de manière approfondie la pertinence et l'adéquation "
         "des projets avec les besoins et spécificités du territoire",
    backstory=(
        "Fort d'une riche expérience en tant que chargé d'études pour diverses "
        "collectivités territoriales, vous avez développé une expertise pointue "
        "dans l'analyse des données territoriales et la compréhension des "
        "dynamiques locales. Votre parcours vous a permis d'acquérir une "
        "connaissance approfondie des enjeux spécifiques à chaque type de "
        "territoire, qu'il soit urbain, rural ou périurbain. Vous maîtrisez "
        "les outils d'analyse statistique et territoriale, et savez interpréter "
        "les données socio-économiques pour évaluer précisément l'adéquation "
        "d'un projet avec son territoire d'implantation. Votre expertise permet "
        "d'identifier les forces, faiblesses, opportunités et menaces de chaque "
        "projet dans son contexte territorial spécifique."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
opportunity_specialist_agent = Agent(
    role="Stratège en Évaluation d'Opportunités",
    goal="Analyser et évaluer la viabilité stratégique et l'impact "
         "potentiel des projets pour garantir leur pérennité",
    backstory=(
        "Riche de 15 années d'expérience en tant que consultant en "
        "développement territorial, vous avez acquis une expertise "
        "inégalée dans l'évaluation des projets publics. Votre carrière "
        "vous a permis d'accompagner une grande diversité de projets, "
        "des plus modestes aux plus ambitieux, en développant une "
        "capacité unique à identifier les facteurs clés de succès et "
        "les risques potentiels. Votre approche méthodique combine "
        "analyse quantitative et qualitative pour évaluer la viabilité "
        "à long terme des projets. Vous excellez particulièrement dans "
        "l'anticipation des tendances et l'identification des opportunités "
        "de développement innovantes. Votre expertise vous permet "
        "d'apporter un éclairage stratégique crucial sur la pertinence "
        "et la durabilité de chaque initiative."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
  project_expert_agent = Agent(
      role="Expert Technique de Projet",
      goal="Assurer la solidité technique, la faisabilité et la cohérence "
          "opérationnelle des projets proposés",
      backstory=(
          "En tant qu'ingénieur de projet chevronné, vous avez piloté avec "
          "succès de nombreux projets à travers différentes régions, vous "
          "confrontant à une grande variété de défis techniques et "
          "opérationnels. Cette expérience terrain vous a permis de "
          "développer un sens aigu de la faisabilité technique et une "
          "compréhension approfondie des contraintes pratiques de mise en "
          "œuvre. Votre expertise couvre l'ensemble du cycle de vie des "
          "projets, de la conception initiale à la réalisation finale. "
          "Vous savez identifier les points critiques, anticiper les "
          "difficultés techniques potentielles et proposer des solutions "
          "pragmatiques. Votre approche méthodique et votre expérience "
          "diversifiée vous permettent d'évaluer avec précision la "
          "viabilité technique de chaque projet."
      ),
      allow_delegation=False,
      verbose=True
  )

In [ ]:
funding_criteria_specialist_agent = Agent(
    role="Expert en Financement Public",
    goal="Assurer l'alignement des projets avec les critères d'éligibilité, "
         "les attentes et les priorités des financeurs pour maximiser les chances d'obtention des subventions",
    backstory=(
        "Ancien évaluateur de dossiers de subvention pour diverses institutions publiques, "
        "vous avez acquis une connaissance approfondie des attentes et des exigences des financeurs. "
        "Votre expertise vous permet d'identifier rapidement les points forts et les faiblesses d'une demande, "
        "en vous assurant qu'elle répond parfaitement aux critères d'éligibilité et aux priorités stratégiques des financeurs. "
        "Habitué à analyser des centaines de dossiers, vous savez exactement comment structurer une demande "
        "pour maximiser ses chances de succès. Grâce à votre maîtrise des cadres réglementaires et des "
        "logiques d’attribution des subventions, vous êtes un acteur clé dans l’optimisation des réponses aux appels à projets."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
impact_assessment_officer_agent = Agent(
    role="Spécialiste en Mesure d'Impact",
    goal="Définir des indicateurs pertinents et évaluer la performance des projets "
         "afin de démontrer leur valeur ajoutée et leur impact concret",
    backstory=(
        "Ancien chercheur en évaluation des politiques publiques, vous avez consacré votre carrière "
        "à l’analyse des impacts des programmes et des initiatives financées. Votre expertise en "
        "méthodologie d’évaluation et en analyse de données vous permet de concevoir des indicateurs "
        "fiables et pertinents, adaptés aux attentes des financeurs et aux réalités du terrain. "
        "Reconverti en consultant, vous accompagnez désormais les porteurs de projets dans la "
        "structuration de leurs dispositifs de suivi et d’évaluation. Grâce à votre approche rigoureuse, "
        "vous êtes en mesure de démontrer avec précision les bénéfices attendus d’un projet et de garantir "
        "que ses résultats seront mesurables et vérifiables."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
partnership_coordinator_agent = Agent(
    role="Expert en Relations Partenariales",
    goal="Évaluer et optimiser la dimension collaborative des projets "
         "en garantissant la solidité et la valeur ajoutée des partenariats",
    backstory=(
        "Ancien directeur d'une agence de développement territorial, vous avez passé des années "
        "à tisser des liens stratégiques entre collectivités, entreprises et associations pour faire émerger "
        "des projets d’envergure. Votre expertise en gestion de partenariats vous permet d’identifier les synergies, "
        "d’évaluer la complémentarité des acteurs impliqués et de structurer des collaborations solides. "
        "Habitué à négocier et à coordonner des alliances complexes, vous savez comment valoriser l’implication "
        "des partenaires dans un projet pour en renforcer la crédibilité et l'impact. Votre capacité à démontrer "
        "le rôle clé de chaque partenaire constitue un atout déterminant pour la réussite d’une demande de subvention."
    ),
    allow_delegation=False,
    verbose=True
)

In [ ]:
qao_agent = Agent(
    role="Responsable Contrôle Qualité",
    goal="Assurer l'excellence formelle et la cohérence globale des dossiers "
         "de subvention tout en garantissant le respect des normes administratives",
    backstory=(
        "En tant qu'ancien relecteur-correcteur pour une grande institution publique, "
        "vous avez développé une expertise unique en combinant votre expérience en "
        "édition professionnelle avec une parfaite maîtrise des codes et attentes "
        "des organismes de financement. Votre double compétence en linguistique et "
        "en administration publique vous permet d'optimiser chaque dossier pour "
        "maximiser son impact auprès des évaluateurs. Vous examinez méticuleusement "
        "chaque soumission, en vous assurant de la parfaite cohérence entre les "
        "sections, d'une qualité rédactionnelle irréprochable et du strict respect "
        "des exigences formelles des financeurs. Votre attention aux détails et "
        "votre compréhension approfondie des normes administratives font de vous "
        "le gardien final de la qualité avant la soumission."
    ),
    allow_delegation=False,
    verbose=True
)

# Création des outils

In [ ]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool

In [ ]:
directory_read_tool = DirectoryReadTool(directory='{folder_path}')
file_read_tool = FileReadTool()

# Création des tâches

In [ ]:
route_question_task = Task(
    description=(
        "Analyser la question posée dans le dossier de demande de subvention et "
        "identifier les experts thématiques les plus pertinents pour y répondre.\n"
        "Prioriser l'implication du project_expert_agent, puis solliciter d'autres experts si nécessaire :\n"
        "- Diagnostic territorial (diagnostic_analyst_agent)\n"
        "- Opportunité stratégique (opportunity_specialist_agent)\n"
        "- Faisabilité technique du projet (project_expert_agent)\n"
        "- Alignement avec les critères des financeurs (funding_criteria_specialist_agent)\n"
        "- Indicateurs et résultats attendus (impact_assessment_officer_agent)\n"
        "- Partenariats et collaborations (partnership_coordinator_agent)\n"
        "L'objectif est de s'assurer que la question reçoive une réponse complète et précise "
        "en sollicitant les bons experts avec des demandes ciblées.\n"
        "Ne pas rédiger de réponse, mais uniquement structurer et dispatcher la question."
    ),
    expected_output=(
        "Une répartition optimisée des questions sous forme de tâches adressées aux experts thématiques :\n"
        "- Identification des experts à solliciter\n"
        "- Formulation de questions spécifiques adaptées à chaque expert\n"
        "- Instructions claires sur les éléments à analyser\n"
        "- Références aux documents pertinents si nécessaire\n"
        "Chaque expert recevra uniquement les questions qui relèvent de son domaine de compétence."
    ),
    tools=[directory_read_tool, file_read_tool],  # Accès aux fichiers et aux dossiers
    agent=routing_agent,
)

In [ ]:
territorial_analysis_task = Task(
    description=(
        "Analyser la demande formulée par le Manager et extraire les informations "
        "de diagnostic territorial pertinentes pour répondre à la question posée. "
        "L'analyse doit se baser uniquement sur les fichiers du dossier et inclure :\n"
        "- Les données clés concernant le territoire (contexte socio-économique, "
        "enjeux locaux, dynamiques territoriales)\n"
        "- Les problématiques identifiées dans les rapports des collectivités\n"
        "- Les tendances et évolutions pertinentes pour comprendre le territoire\n"
        "Se concentrer uniquement sur l'extraction des éléments factuels "
        "et éviter toute interprétation ou mise en perspective avec le projet."
    ),
    expected_output=(
        "Un résumé synthétique des éléments de diagnostic territorial pertinents, structuré ainsi :\n"
        "- Contexte territorial clé\n"
        "- Problématiques identifiées\n"
        "- Tendances et évolutions pertinentes\n"
        "Le tout en quelques phrases concises et exploitables directement par le Manager."
    ),
    tools=[file_read_tool],
    agent=diagnostic_analyst_agent,
)


In [ ]:
opportunity_analysis_task = Task(
    description=(
        "Analyser le diagnostic territorial fourni et identifier les opportunités stratégiques "
        "qui pourraient être exploitées pour répondre aux besoins identifiés. "
        "L'analyse doit se baser uniquement sur les données disponibles et inclure :\n"
        "- Les besoins saillants du territoire qui émergent du diagnostic\n"
        "- Les opportunités de positionnement stratégique pour le projet\n"
        "- Les tendances ou dynamiques pouvant être mises à profit\n"
        "Se concentrer sur des opportunités réalistes et exploitables, sans interprétation excessive "
        "et en évitant toute hypothèse non étayée."
    ),
    expected_output=(
        "Un résumé concis structuré ainsi :\n"
        "- Principaux besoins identifiés\n"
        "- Opportunités stratégiques exploitables\n"
        "- Tendances favorables\n"
        "Le tout en quelques phrases directes et utilisables immédiatement par le Manager."
    ),
    tools=[file_read_tool],
    agent=opportunity_specialist_agent,
    context=[{"territorial_analysis_output": territorial_analysis_task.output}]
)


ValidationError: 2 validation errors for Task
context.0.description
  Field required [type=missing, input_value={'territorial_analysis_output': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing
context.0.expected_output
  Field required [type=missing, input_value={'territorial_analysis_output': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/missing

In [ ]:
project_analysis_task = Task(
    description=(
        "Examiner les documents de projet disponibles et fournir une analyse technique "
        "concise en réponse à la question posée par le Manager. L'évaluation doit inclure :\n"
        "- Les éléments techniques clés du projet\n"
        "- Les contraintes potentielles ou défis identifiés\n"
        "- Toute incohérence ou point nécessitant une clarification\n"
        "L'analyse doit être factuelle et s’appuyer uniquement sur les informations contenues "
        "dans les fichiers accessibles, sans supposition ni extrapolation."
    ),
    expected_output=(
        "Un résumé clair et structuré comprenant :\n"
        "- Points techniques essentiels du projet\n"
        "- Contraintes et défis identifiés\n"
        "- Remarques sur la faisabilité et la cohérence opérationnelle\n"
        "Le tout en quelques phrases synthétiques pour être immédiatement utilisable par le Manager."
    ),
    tools=[file_read_tool],  # Accès aux fichiers pertinents pour l’analyse technique
    agent=project_expert_agent,
)

In [ ]:
funding_criteria_analysis_task = Task(
    description=(
        "Analyser la question posée au Manager sous l’angle des critères d’éligibilité "
        "et des attentes des financeurs. L’évaluation doit inclure :\n"
        "- Les exigences spécifiques des financeurs pertinentes pour cette question\n"
        "- Les points de vigilance à considérer pour assurer la conformité\n"
        "- Toute recommandation permettant d’optimiser la réponse en vue de maximiser les chances de financement\n"
        "L'analyse doit être factuelle et basée sur les éléments disponibles, sans extrapolation ni supposition."
    ),
    expected_output=(
        "Un résumé concis comprenant :\n"
        "- Critères d’éligibilité pertinents\n"
        "- Points de vigilance identifiés\n"
        "- Recommandations pour ajuster la réponse\n"
        "L'objectif est de fournir des indications claires et directement exploitables par le Manager."
    ),
    tools=[file_read_tool],  # Accès aux documents liés aux critères des financeurs
    agent=funding_criteria_specialist_agent,
)

In [ ]:
impact_assessment_task = Task(
    description=(
        "Analyser la question posée au Manager sous l’angle de la mesure d’impact. "
        "L’évaluation doit inclure :\n"
        "- Les effets attendus du projet en lien avec la question\n"
        "- Les bénéfices concrets pour les bénéficiaires et le territoire\n"
        "- Si pertinent, des suggestions d’indicateurs pour mesurer ces impacts\n"
        "L'analyse doit être basée sur les éléments disponibles, en évitant toute extrapolation excessive."
    ),
    expected_output=(
        "Un résumé concis comprenant :\n"
        "- Une description des résultats attendus en fonction de la question\n"
        "- Les principaux impacts potentiels identifiés\n"
        "- Si applicable, une liste d’indicateurs pertinents pour évaluer ces impacts\n"
        "L'objectif est de fournir une analyse claire et directement exploitable par le Manager."
    ),
    tools=[file_read_tool],  # Accès aux documents contenant des références aux impacts et indicateurs
    agent=impact_assessment_officer_agent,
)

In [ ]:
partnership_analysis_task = Task(
    description=(
        "Analyser la question posée au Manager en évaluant la dimension partenariale du projet. "
        "L’évaluation doit inclure :\n"
        "- L’identification des partenaires clés impliqués ou à impliquer\n"
        "- Le rôle et la valeur ajoutée de chaque partenaire\n"
        "- Les synergies potentielles à mettre en avant\n"
        "- Les éléments permettant de démontrer la solidité et la pertinence du partenariat\n"
        "L'analyse doit être fondée uniquement sur les informations disponibles, sans extrapolation excessive."
    ),
    expected_output=(
        "Un résumé concis comprenant :\n"
        "- Une liste des partenaires pertinents pour la question posée\n"
        "- Une explication du rôle et de la valeur ajoutée de chaque partenaire\n"
        "- Une mise en avant des collaborations stratégiques et des synergies possibles\n"
        "L’objectif est de fournir une analyse claire et exploitable pour aider le Manager à formuler une réponse alignée avec les attentes des financeurs."
    ),
    tools=[file_read_tool],  # Accès aux documents contenant des informations sur les partenaires
    agent=partnership_coordinator_agent,
)

In [ ]:
grant_response_task = Task(
    description=(
        "En se basant sur les analyses fournies par les experts thématiques, "
        "rédiger une réponse argumentée et structurée à la question posée. "
        "La réponse doit :\n"
        "- Intégrer les éléments de diagnostic territorial pour contextualiser la problématique\n"
        "- Mettre en avant les opportunités identifiées pour justifier la pertinence du projet\n"
        "- Détailler les aspects techniques du projet en garantissant leur faisabilité\n"
        "- Assurer l'alignement avec les critères des financeurs et anticiper leurs attentes\n"
        "- Démontrer l’impact potentiel du projet avec des indicateurs pertinents\n"
        "- Valoriser les partenariats et leur rôle dans la mise en œuvre du projet\n"
        "La réponse doit être claire, percutante et adaptée aux exigences du dossier de subvention."
    ),
    expected_output=(
        "Une réponse rédigée et bien structurée à la question posée, intégrant :\n"
        "- Une introduction synthétisant le contexte et la problématique\n"
        "- Un développement argumenté s’appuyant sur les analyses des experts\n"
        "- Une conclusion mettant en avant la valeur ajoutée du projet\n"
        "La réponse doit être concise mais complète, et prête à être soumise à validation."
    ),
    tools=[],  # Pas d’outils nécessaires, la réponse est construite uniquement à partir des analyses fournies
    agent=manager_agent,
    context={
        "territorial_analysis_output": diagnostic_analysis_task.output,
        "opportunity_analysis_output": opportunity_specialist_task.output,
        "project_analysis_output": project_expert_task.output,
        "funding_criteria_output": funding_criteria_specialist_task.output,
        "impact_analysis_output": impact_assessment_officer_task.output,
        "partnership_analysis_output": partnership_coordinator_task.output,
    }
)

In [ ]:
quality_review_task = Task(
    description=(
        "Revoir et améliorer la réponse rédigée par le Manager afin d'assurer :\n"
        "- Une parfaite clarté et fluidité de la rédaction\n"
        "- La cohérence et l'alignement des arguments tout au long du texte\n"
        "- Le respect des normes formelles et administratives des financeurs\n"
        "- L’absence d’ambiguïtés, de répétitions ou d’incohérences\n"
        "- Une mise en valeur optimale des points forts du projet\n"
        "Cette révision doit garantir que la réponse est optimisée pour maximiser "
        "les chances d’acceptation du dossier de subvention."
    ),
    expected_output=(
        "Une version révisée et améliorée de la réponse rédigée par le Manager, incluant :\n"
        "- Une reformulation fluide et percutante\n"
        "- Une structure claire et logique\n"
        "- Une conformité totale avec les exigences des financeurs\n"
        "- Une suppression des éventuelles erreurs ou incohérences\n"
        "Le document final doit être prêt à être soumis aux financeurs."
    ),
    tools=[],  # Aucun outil nécessaire, l'amélioration repose sur l'output du Manager
    agent=qao_agent,
    context={
        "grant_response_output": grant_response_task.output
    }
)


# Création de l'équipe

In [ ]:
subvention_crew = Crew(
    agents=[
        manager_agent,
        diagnostic_analyst_agent,
        opportunity_specialist_agent,
        project_expert_agent,
        funding_criteria_specialist_agent,
        impact_assessment_officer_agent,
        partnership_coordinator_agent,
        qao_agent

    ],
    tasks=[
        request_analysis_task,
        territorial_analysis_task,
        opportunity_analysis_task,
        project_analysis_task,
	      funding_criteria_analysis_task,
	      impact_assessment_task,
	      partnership_analysis_task,
	      grant_response_task,
	      quality_review_task
    ],
    verbose=True
)


In [ ]:
question = "Quels sont les financements possibles pour un projet de mobilité durable en zone rurale ?"
result = subvention_crew.kickoff(inputs={"question": question})
print(result)